# Скачаем датасет данных с Twitter Sentiment

In [1]:
! mkdir -p /home/jovyan/data
! mkdir -p /home/jovyan/models

In [2]:
! wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip -O /home/jovyan/data/sentiment.zip

--2020-01-25 13:48:35--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2020-01-25 13:48:36--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘/home/jovyan/data/sentiment.zip’

/home/jovyan/data/s 100%[===================>]  77.59M  4.30MB/s    in 34s     

2020-01-25 13:49:12 (2.31 MB/s) - ‘/home/jovyan/data/sentiment.zip’ saved [81363704/81363704]



In [3]:
! cd /home/jovyan/data && unzip sentiment.zip

Archive:  sentiment.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [4]:
! ls -la /home/jovyan/data

total 312744
drwxr-sr-x 2 jovyan users       114 Jan 25 13:49 .
drwsrwsr-x 1 jovyan users        94 Jan 25 13:49 ..
-rw-r--r-- 1 jovyan users  81363704 Apr 22  2012 sentiment.zip
-rw-r--r-- 1 jovyan users     74326 Mar  4  2010 testdata.manual.2009.06.14.csv
-rw-r--r-- 1 jovyan users 238803811 Mar  4  2010 training.1600000.processed.noemoticon.csv


In [5]:
! head -n 5 /home/jovyan/data/training.1600000.processed.noemoticon.csv

"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
"0","1467810672","Mon Apr 06 22:19:49 PDT 2009","NO_QUERY","scotthamilton","is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
"0","1467810917","Mon Apr 06 22:19:53 PDT 2009","NO_QUERY","mattycus","@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds"
"0","1467811184","Mon Apr 06 22:19:57 PDT 2009","NO_QUERY","ElleCTF","my whole body feels itchy and like its on fire "
"0","1467811193","Mon Apr 06 22:19:57 PDT 2009","NO_QUERY","Karoli","@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "


# Читаем датасет с помощью Spark

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('twitter-sentiment').getOrCreate()

print("Spark context started")

Spark context started


In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType

schema = StructType([
    StructField("target", IntegerType(), True),
    StructField("id", LongType(), True),
    StructField("raw_timestamp", StringType(), True),
    StructField("query_status", StringType(), True),
    StructField("author", StringType(), True),
    StructField("tweet", StringType(), True)
])
    
data_path = "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

raw_sentiment = spark.read.csv(data_path,header=False,schema=schema) \
    .selectExpr("(case when target=4 then 1 else 0 end) as target","tweet")



raw_sentiment.groupBy("target").count().show()

+------+------+
|target| count|
+------+------+
|     1|800000|
|     0|800000|
+------+------+



In [8]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 1.5MB 218kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449909 sha256=22c36f951225c02eb57500ab9e5da4a2fc6dc5ce9e60e19e2c95a205e9848d96
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
raw_sentiment_sample = raw_sentiment.sample(fraction=0.005,withReplacement=False,seed=42).toPandas()
X, y = raw_sentiment_sample["tweet"], raw_sentiment_sample["target"]

print("Dataset size is: %i" % X.size)


Dataset size is: 8073


In [15]:
import pickle as pkl

def save_model(model,model_path):
    with open(model_path,'wb') as buffer:
        pkl.dump(model,buffer)

def read_model(model_path):
    with open(model_path,'rb') as buffer:
        return pkl.load(buffer)

model_path = "/home/jovyan/tweet_sentiment.mdl"
save_model(grid_search.best_estimator_,model_path)

NameError: name 'grid_search' is not defined

In [11]:
model_object = read_model(model_path)
model_object

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/tweet_sentiment.mdl'

In [ ]:
%matplotlib inline
import pandas as pd

pd.Series(model_object.predict_proba(X)[:,1]).hist(figsize=(20,10))

In [ ]:
spark.stop()